In [ ]:
import numpy as np
import matplotlib.pyplot as plt

"""
v_t + (v.nabla) v = nu * nabla^2 v + nabla P
div(v) = 0

"""


def poisson(rho, kSq_inv):
    """"""
    V_hat = -(np.fft.fftn(rho)) * kSq_inv
    V = np.real(np.fft.ifftn(V_hat))
    return V


def diffusion(v, dt, nu, kSq):
    """"""
    v_hat = (np.fft.fftn(v)) / (1.0 + dt * nu * kSq)
    v = np.real(np.fft.ifftn(v_hat))
    return v


def grad(v, kx, ky):
    """"""
    v_hat = np.fft.fftn(v)
    dvx = np.real(np.fft.ifftn(1j * kx * v_hat))
    dvy = np.real(np.fft.ifftn(1j * ky * v_hat))
    return dvx, dvy


def div(vx, vy, kx, ky):
    """"""
    dvx_x = np.real(np.fft.ifftn(1j * kx * np.fft.fftn(vx)))
    dvy_y = np.real(np.fft.ifftn(1j * ky * np.fft.fftn(vy)))
    return dvx_x + dvy_y


def curl(vx, vy, kx, ky):
    """"""
    dvx_y = np.real(np.fft.ifftn(1j * ky * np.fft.fftn(vx)))
    dvy_x = np.real(np.fft.ifftn(1j * kx * np.fft.fftn(vy)))
    return dvy_x - dvx_y


def dealias_23(f, dealias):
    """"""
    f_hat = dealias * np.fft.fftn(f)
    return np.real(np.fft.ifftn(f_hat))


import os
import imageio
from PIL import Image as PILImage
import glob
from IPython.display import Image as IPyImage
from mpl_toolkits.axes_grid1 import make_axes_locatable


# Simulation A
N = 600       # resolution
dt = 0.01    # timestep
tEnd = 1    # total time
tOut = 0.01   # draw every 0.01s
nu = 0.01   # viscosity
t = 0.0       # initial time

# Domain [0,1] x [0,1]
L = 1
xlin = np.linspace(0, L, num=N + 1)  
xlin = xlin[0:N]  
xx, yy = np.meshgrid(xlin, xlin)

# Initial Condition (vortex)
vx = -np.sin(2 * np.pi * yy)
vy = np.sin(2 * np.pi * xx * 2)

# Fourier Space Variables
klin = 2.0 * np.pi / L * np.arange(-N / 2, N / 2)
kmax = np.max(klin)
kx, ky = np.meshgrid(klin, klin)
kx = np.fft.ifftshift(kx)
ky = np.fft.ifftshift(ky)
kSq = kx**2 + ky**2
kSq_inv = np.zeros_like(kSq)
kSq_inv[kSq != 0] = 1.0 / kSq[kSq != 0]
kSq_inv[kSq == 0] = 1.0

# dealias with the 2/3 rule
dealias_mask = (np.abs(kx) < (2.0 / 3.0) * kmax) & (np.abs(ky) < (2.0 / 3.0) * kmax)

# number of timesteps
Nt = int(np.ceil(tEnd / dt))

# prep figure
fig = plt.figure(figsize=(4, 4), dpi=80)
# Prepare output folders
output_folder = "output_snapshots_2"
gif_filename = "navier-stokes-spectral-presentation_2.gif"
os.makedirs(output_folder, exist_ok=True)

# For storing frames for GIF
wz_frames = []
outputCount = 1

# Main Loop
for i in range(Nt):
    # Advection: rhs = -(v.grad)v
    dvx_x, dvx_y = grad(vx, kx, ky)
    dvy_x, dvy_y = grad(vy, kx, ky)

    rhs_x = -(vx * dvx_x + vy * dvx_y)
    rhs_y = -(vx * dvy_x + vy * dvy_y)

    rhs_x = dealias_23(rhs_x, dealias_mask)
    rhs_y = dealias_23(rhs_y, dealias_mask)

    vx += dt * rhs_x
    vy += dt * rhs_y

    # Poisson solve for pressure
    div_rhs = div(rhs_x, rhs_y, kx, ky)
    P = poisson(div_rhs, kSq_inv)
    dPx, dPy = grad(P, kx, ky)

    # Correction (to eliminate divergence component of velocity)
    vx += -dt * dPx
    vy += -dt * dPy

    # Diffusion solve (implicit)
    vx = diffusion(vx, dt, nu, kSq)
    vy = diffusion(vy, dt, nu, kSq)
    # div_v = np.fft.ifft2(1j * kx * np.fft.fft2(vx) + 1j * ky * np.fft.fft2(vy)).real
    # print(f"Step {i}, max|div(v)| = {np.abs(div_v).max():.2e}")

    wz = curl(vx, vy, kx, ky)
    t += dt

    if i % int(tOut / dt) == 0 or i == Nt - 1:
        # Compute stream function from vorticity: ∇²ψ = -ω ⇒ ψ̂ = -ω̂ / k²
        wz_hat = np.fft.fft2(wz)
        psi_hat = -wz_hat * kSq_inv
        psi = np.fft.ifft2(psi_hat).real

        # Create vertical multiplot: vorticity and stream function
        fig_gif, axes = plt.subplots(2, 1, figsize=(7, 9), dpi=150)

        # === Vorticity Plot ===
        im0 = axes[0].imshow(wz, cmap="viridis", origin="lower", extent=[0, L, 0, L], vmin=-10, vmax=10)
        axes[0].set_xlabel("x [units of length]", fontsize=12)
        axes[0].set_ylabel("y [units of length]", fontsize=12)
        axes[0].set_title(f"Vorticity ω at t = {t:.3f} [s]", fontsize=14)
        divider0 = make_axes_locatable(axes[0])
        cax0 = divider0.append_axes("right", size="5%", pad=0.05)
        cbar0 = fig_gif.colorbar(im0, cax=cax0)
        cbar0.set_label("ω [1/s]", fontsize=12)

        # === Stream Function Plot ===
        levels = np.linspace(np.min(psi), np.max(psi), 50)
        cont = axes[1].contourf(xx, yy, psi, levels=levels, cmap="coolwarm")
        axes[1].set_xlabel("x [units of length]", fontsize=12)
        axes[1].set_ylabel("y [units of length]", fontsize=12)
        axes[1].set_title(f"Stream Function ψ at t = {t:.3f} [s]", fontsize=14)
        divider1 = make_axes_locatable(axes[1])
        cax1 = divider1.append_axes("right", size="5%", pad=0.05)
        cbar1 = fig_gif.colorbar(cont, cax=cax1)
        cbar1.set_label("ψ [length²/s]", fontsize=12)

        for ax in axes:
            ax.tick_params(axis='both', labelsize=10)

        plt.tight_layout()
        fig_gif.canvas.draw()

        # Convert full scientific frame to image array
        img_array = np.frombuffer(fig_gif.canvas.tostring_rgb(), dtype=np.uint8)
        img_array = img_array.reshape(fig_gif.canvas.get_width_height()[::-1] + (3,))
        wz_frames.append(img_array)
        # Save multiplot snapshot as PNG
        snapshot_dir = output_folder + '_1'
        os.makedirs(snapshot_dir, exist_ok=True)
        snapshot_filename = os.path.join(snapshot_dir, f"multiplot_snapshot_{outputCount:04d}.png")
        fig_gif.savefig(snapshot_filename, bbox_inches='tight')


        plt.close(fig_gif)
        outputCount += 1
        # Save GIF
if wz_frames:
    imageio.mimsave(gif_filename, wz_frames, fps=10)  # adjust fps if needed

# Display GIF
# IPyImage(filename=gif_filename)

In [10]:
import os
import imageio
from PIL import Image as PILImage
import glob
from IPython.display import Image as IPyImage
from mpl_toolkits.axes_grid1 import make_axes_locatable


# Simulation A
N = 600       # resolution
dt = 0.01    # timestep
tEnd = 1    # total time
tOut = 0.01   # draw every 0.01s
nu = 0.01   # viscosity
t = 0.0       # initial time

# Domain [0,1] x [0,1]
L = 1
xlin = np.linspace(0, L, num=N + 1)  
xlin = xlin[0:N]  
xx, yy = np.meshgrid(xlin, xlin)

# Initial Condition (vortex)
vx = -np.sin(2 * np.pi * yy)
vy = np.sin(2 * np.pi * xx * 2)

# Fourier Space Variables
klin = 2.0 * np.pi / L * np.arange(-N / 2, N / 2)
kmax = np.max(klin)
kx, ky = np.meshgrid(klin, klin)
kx = np.fft.ifftshift(kx)
ky = np.fft.ifftshift(ky)
kSq = kx**2 + ky**2
kSq_inv = np.zeros_like(kSq)
kSq_inv[kSq != 0] = 1.0 / kSq[kSq != 0]
kSq_inv[kSq == 0] = 1.0

# dealias with the 2/3 rule
dealias_mask = (np.abs(kx) < (2.0 / 3.0) * kmax) & (np.abs(ky) < (2.0 / 3.0) * kmax)

# number of timesteps
Nt = int(np.ceil(tEnd / dt))

# prep figure
fig = plt.figure(figsize=(4, 4), dpi=80)
# Prepare output folders
output_folder = "output_snapshots_2"
gif_filename = "navier-stokes-spectral-presentation_2.gif"
os.makedirs(output_folder, exist_ok=True)

# For storing frames for GIF
wz_frames = []
outputCount = 1

# Main Loop
for i in range(Nt):
    # Advection: rhs = -(v.grad)v
    dvx_x, dvx_y = grad(vx, kx, ky)
    dvy_x, dvy_y = grad(vy, kx, ky)

    rhs_x = -(vx * dvx_x + vy * dvx_y)
    rhs_y = -(vx * dvy_x + vy * dvy_y)

    rhs_x = dealias_23(rhs_x, dealias_mask)
    rhs_y = dealias_23(rhs_y, dealias_mask)

    vx += dt * rhs_x
    vy += dt * rhs_y

    # Poisson solve for pressure
    div_rhs = div(rhs_x, rhs_y, kx, ky)
    P = poisson(div_rhs, kSq_inv)
    dPx, dPy = grad(P, kx, ky)

    # Correction (to eliminate divergence component of velocity)
    vx += -dt * dPx
    vy += -dt * dPy

    # Diffusion solve (implicit)
    vx = diffusion(vx, dt, nu, kSq)
    vy = diffusion(vy, dt, nu, kSq)
    # div_v = np.fft.ifft2(1j * kx * np.fft.fft2(vx) + 1j * ky * np.fft.fft2(vy)).real
    # print(f"Step {i}, max|div(v)| = {np.abs(div_v).max():.2e}")

    wz = curl(vx, vy, kx, ky)
    t += dt

    if i % int(tOut / dt) == 0 or i == Nt - 1:
        # Compute stream function from vorticity: ∇²ψ = -ω ⇒ ψ̂ = -ω̂ / k²
        wz_hat = np.fft.fft2(wz)
        psi_hat = -wz_hat * kSq_inv
        psi = np.fft.ifft2(psi_hat).real

        # Create vertical multiplot: vorticity and stream function
        fig_gif, axes = plt.subplots(2, 1, figsize=(7, 9), dpi=150)

        # === Vorticity Plot ===
        im0 = axes[0].imshow(wz, cmap="viridis", origin="lower", extent=[0, L, 0, L], vmin=-10, vmax=10)
        axes[0].set_xlabel("x [units of length]", fontsize=12)
        axes[0].set_ylabel("y [units of length]", fontsize=12)
        axes[0].set_title(f"Vorticity ω at t = {t:.3f} [s]", fontsize=14)
        divider0 = make_axes_locatable(axes[0])
        cax0 = divider0.append_axes("right", size="5%", pad=0.05)
        cbar0 = fig_gif.colorbar(im0, cax=cax0)
        cbar0.set_label("ω [1/s]", fontsize=12)

        # === Stream Function Plot ===
        levels = np.linspace(np.min(psi), np.max(psi), 50)
        cont = axes[1].contourf(xx, yy, psi, levels=levels, cmap="coolwarm")
        axes[1].set_xlabel("x [units of length]", fontsize=12)
        axes[1].set_ylabel("y [units of length]", fontsize=12)
        axes[1].set_title(f"Stream Function ψ at t = {t:.3f} [s]", fontsize=14)
        divider1 = make_axes_locatable(axes[1])
        cax1 = divider1.append_axes("right", size="5%", pad=0.05)
        cbar1 = fig_gif.colorbar(cont, cax=cax1)
        cbar1.set_label("ψ [length²/s]", fontsize=12)

        for ax in axes:
            ax.tick_params(axis='both', labelsize=10)

        plt.tight_layout()
        fig_gif.canvas.draw()

        # Convert full scientific frame to image array
        img_array = np.frombuffer(fig_gif.canvas.tostring_rgb(), dtype=np.uint8)
        img_array = img_array.reshape(fig_gif.canvas.get_width_height()[::-1] + (3,))
        wz_frames.append(img_array)
        # Save multiplot snapshot as PNG
        snapshot_dir = output_folder + '_1'
        os.makedirs(snapshot_dir, exist_ok=True)
        snapshot_filename = os.path.join(snapshot_dir, f"multiplot_snapshot_{outputCount:04d}.png")
        fig_gif.savefig(snapshot_filename, bbox_inches='tight')


        plt.close(fig_gif)
        outputCount += 1
        # Save GIF
if wz_frames:
    imageio.mimsave(gif_filename, wz_frames, fps=10)  # adjust fps if needed

# Display GIF
# IPyImage(filename=gif_filename)

/tmp/ipykernel_5691/319013605.py:125: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed in 3.10. Use buffer_rgba instead.
  img_array = np.frombuffer(fig_gif.canvas.tostring_rgb(), dtype=np.uint8)


<Figure size 320x320 with 0 Axes>

In [11]:
import os
import imageio
from PIL import Image as PILImage
import glob
from IPython.display import Image as IPyImage
from mpl_toolkits.axes_grid1 import make_axes_locatable


# Simulation parameters
N = 400  # Spatial resolution
t = 0  # current time of the simulation
tEnd = 1.0  # time at which simulation ends
dt = 0.001  # timestep
tOut = 0.01  # draw frequency
nu = 0.04 # viscosity

# Domain [0,1] x [0,1]
L = 1
xlin = np.linspace(0, L, num=N + 1)  
xlin = xlin[0:N]  
xx, yy = np.meshgrid(xlin, xlin)

# Initial Condition (vortex)
vx = -np.sin(2 * np.pi * yy)
vy = np.sin(2 * np.pi * xx * 2)

# Fourier Space Variables
klin = 2.0 * np.pi / L * np.arange(-N / 2, N / 2)
kmax = np.max(klin)
kx, ky = np.meshgrid(klin, klin)
kx = np.fft.ifftshift(kx)
ky = np.fft.ifftshift(ky)
kSq = kx**2 + ky**2
kSq_inv = np.zeros_like(kSq)
kSq_inv[kSq != 0] = 1.0 / kSq[kSq != 0]
kSq_inv[kSq == 0] = 1.0

# dealias with the 2/3 rule
dealias_mask = (np.abs(kx) < (2.0 / 3.0) * kmax) & (np.abs(ky) < (2.0 / 3.0) * kmax)

# number of timesteps
Nt = int(np.ceil(tEnd / dt))

# prep figure
fig = plt.figure(figsize=(4, 4), dpi=80)
# Prepare output folders
output_folder = "output_snapshots_2"
gif_filename = "navier-stokes-spectral-presentation_2.gif"
os.makedirs(output_folder, exist_ok=True)

# For storing frames for GIF
wz_frames = []
outputCount = 1

# Main Loop
for i in range(Nt):
    # Advection: rhs = -(v.grad)v
    dvx_x, dvx_y = grad(vx, kx, ky)
    dvy_x, dvy_y = grad(vy, kx, ky)

    rhs_x = -(vx * dvx_x + vy * dvx_y)
    rhs_y = -(vx * dvy_x + vy * dvy_y)

    rhs_x = dealias_23(rhs_x, dealias_mask)
    rhs_y = dealias_23(rhs_y, dealias_mask)

    vx += dt * rhs_x
    vy += dt * rhs_y

    # Poisson solve for pressure
    div_rhs = div(rhs_x, rhs_y, kx, ky)
    P = poisson(div_rhs, kSq_inv)
    dPx, dPy = grad(P, kx, ky)

    # Correction (to eliminate divergence component of velocity)
    vx += -dt * dPx
    vy += -dt * dPy

    # Diffusion solve (implicit)
    vx = diffusion(vx, dt, nu, kSq)
    vy = diffusion(vy, dt, nu, kSq)
    # div_v = np.fft.ifft2(1j * kx * np.fft.fft2(vx) + 1j * ky * np.fft.fft2(vy)).real
    # print(f"Step {i}, max|div(v)| = {np.abs(div_v).max():.2e}")

    wz = curl(vx, vy, kx, ky)
    t += dt

    if i % int(tOut / dt) == 0 or i == Nt - 1:
        # Compute stream function from vorticity: ∇²ψ = -ω ⇒ ψ̂ = -ω̂ / k²
        wz_hat = np.fft.fft2(wz)
        psi_hat = -wz_hat * kSq_inv
        psi = np.fft.ifft2(psi_hat).real

        # Create vertical multiplot: vorticity and stream function
        fig_gif, axes = plt.subplots(2, 1, figsize=(7, 9), dpi=150)

        # === Vorticity Plot ===
        im0 = axes[0].imshow(wz, cmap="viridis", origin="lower", extent=[0, L, 0, L], vmin=-10, vmax=10)
        axes[0].set_xlabel("x [units of length]", fontsize=12)
        axes[0].set_ylabel("y [units of length]", fontsize=12)
        axes[0].set_title(f"Vorticity ω at t = {t:.3f} [s]", fontsize=14)
        divider0 = make_axes_locatable(axes[0])
        cax0 = divider0.append_axes("right", size="5%", pad=0.05)
        cbar0 = fig_gif.colorbar(im0, cax=cax0)
        cbar0.set_label("ω [1/s]", fontsize=12)

        # === Stream Function Plot ===
        levels = np.linspace(np.min(psi), np.max(psi), 50)
        cont = axes[1].contourf(xx, yy, psi, levels=levels, cmap="coolwarm")
        axes[1].set_xlabel("x [units of length]", fontsize=12)
        axes[1].set_ylabel("y [units of length]", fontsize=12)
        axes[1].set_title(f"Stream Function ψ at t = {t:.3f} [s]", fontsize=14)
        divider1 = make_axes_locatable(axes[1])
        cax1 = divider1.append_axes("right", size="5%", pad=0.05)
        cbar1 = fig_gif.colorbar(cont, cax=cax1)
        cbar1.set_label("ψ [length²/s]", fontsize=12)

        for ax in axes:
            ax.tick_params(axis='both', labelsize=10)

        plt.tight_layout()
        fig_gif.canvas.draw()

        # Convert full scientific frame to image array
        img_array = np.frombuffer(fig_gif.canvas.tostring_rgb(), dtype=np.uint8)
        img_array = img_array.reshape(fig_gif.canvas.get_width_height()[::-1] + (3,))
        wz_frames.append(img_array)
        # Save multiplot snapshot as PNG
        snapshot_dir = output_folder + '_3'
        os.makedirs(snapshot_dir, exist_ok=True)
        snapshot_filename = os.path.join(snapshot_dir, f"multiplot_snapshot_{outputCount:04d}.png")
        fig_gif.savefig(snapshot_filename, bbox_inches='tight')


        plt.close(fig_gif)
        outputCount += 1
        # Save GIF
if wz_frames:
    imageio.mimsave(gif_filename, wz_frames, fps=10)  # adjust fps if needed

# Display GIF
# IPyImage(filename=gif_filename)

/tmp/ipykernel_5691/2334643634.py:125: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed in 3.10. Use buffer_rgba instead.
  img_array = np.frombuffer(fig_gif.canvas.tostring_rgb(), dtype=np.uint8)


<Figure size 320x320 with 0 Axes>